[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/philzook58/knuckledragger/blob/main/examples/nng.ipynb)
Run the following cells to download and install Knuckledragger.

In [23]:
#%%bash
#git clone https://github.com/philzook58/knuckledragger.git
#cd knuckledragger
#python3 -m pip install -e .

If the following cell fails after first installation, you may need to restart the notebook runtime

In [24]:
import kdrag as kd
import kdrag.smt as smt # smt is essentially a reexport of z3

#Or for short:
#from kdrag.all import *

# Inductive Datatypes

The [Peano axioms](https://en.wikipedia.org/wiki/Peano_axioms) are a way to build the natural numbers from a few simple rules.

The natural numbers are generated by a term `Zero` and a function `Succ` that takes a natural number and returns the next one.

We can declare an inductive datatype for `Nat` in a manner similar to that avaiable in functional programming languages like OCaml and Haskell.

This mechanism is also a simple wrapping of the [Datatype declaration from z3](https://z3prover.github.io/api/html/classz3py_1_1_datatype.html)

In [25]:
Nat = kd.Inductive("MyNat")
Zero = Nat.declare("Zero")
Succ = Nat.declare("Succ", ("pred", Nat))
Nat = Nat.create()

Being an inductive datatype has implicit property that the constructors are disjoint and injective. Since knuckeldragger is built on Z3 and Z3 deals with this intrinsically, these lemmas require no user input but also are not of that much use.

In [26]:
n,m = smt.Consts("n m", Nat)

In [27]:
zero_succ_disjoint = kd.prove(smt.ForAll([n], Nat.Zero != Nat.Succ(n)))
zero_succ_disjoint

|- ForAll(n, Zero != Succ(n))

Note that the object returned by `kd.prove` is a `kd.Proof` object, not an `smt.BoolRef` object.

In [28]:
type(zero_succ_disjoint)

kdrag.kernel.Proof

In [29]:
type(zero_succ_disjoint.thm)

z3.z3.QuantifierRef

If you try to create a false lemma or something just plain too difficult for the automation without help, `kd.prove` will raise an exception.

In [30]:
try:
    succ_succ_disjoint = kd.prove(smt.ForAll([n,m], Nat.Succ(m) == Nat.Succ(n)), timeout=10)
except kd.kernel.LemmaError as e:
    print(e)

(Succ(m) == Succ(n), 'Countermodel', [m = Succ(Zero), n = Zero])


A useful helper for statements of the form $\forall x, precond(x) => conc(x)$ is the `kd.QForAll` quantifier, which allows you to insert an arbitrary number of preconditions.

In [31]:
succ_inj = kd.prove(kd.QForAll([n,m], Nat.Succ(n) == Nat.Succ(m), n == m))
succ_inj

|- ForAll([n, m], Implies(Succ(n) == Succ(m), n == m))

# Definitions
Knuckledragger offers a definition mechanism.

One possibility is to define numerals at the python level. This works and is quite useful.

In [32]:
one = Nat.Succ(Nat.Zero)
two = Nat.Succ(one)
three = Nat.Succ(two)
one,two,three

(Succ(Zero), Succ(Succ(Zero)), Succ(Succ(Succ(Zero))))

What it doesn't allow is building a hierarchy of abstractions. Sometimes you want to hide the details of a definition and manipulate them at an abstract level.


In [33]:
one = kd.define("one", [], Nat.Succ(Nat.Zero))
two = kd.define("two", [], Nat.Succ(one))
three = kd.define("three", [], Nat.Succ(two))
one,two,three

(one, two, three)

These new definitions produce a definitional theorem linking them to their definition

In [34]:
one.defn,two.defn,three.defn

(|- one == Succ(Zero), |- two == Succ(one), |- three == Succ(two))

In order to prove this simple lemma, Z3 will need access to two of the definitions to unfold.

In [35]:
try:
    kd.prove(three == Nat.Succ(Nat.Succ(one)))
except kd.kernel.LemmaError as e:
    print(e)

(three == Succ(Succ(one)), 'Countermodel', [three = Succ(Succ(Succ(Zero))), one = Zero])


In [36]:
kd.prove(three == Nat.Succ(Nat.Succ(one)), by=[two.defn,three.defn])

|- three == Succ(Succ(one))

# Addition

We can define an addition function on natural numbers by recursion.

Knuckledragger does not currently have protection to avoid circular definitions, so be careful.


In [37]:
add1 = smt.Function("add1", Nat, Nat, Nat)
add1 = kd.define("add1", [n,m], smt.If(n == Nat.Zero, m, Nat.Succ(add1(n.pred, m))))

A similar definition can use the helper function `kd.cond` which allows you to list cases. This will expand into a chained `smt.`If` statements.

In [38]:
add = smt.Function("add", Nat, Nat, Nat)
add = kd.define("add", [n,m], kd.cond(
    (n.is_Zero, m),
    (n.is_Succ, Nat.Succ(add(n.pred, m)))
))

It is extremely nice to have overloading available for operators. Knuckledragger offers a Z3 sort dispatch mechanism modelled after python's [singledispatch](https://docs.python.org/3/library/functools.html#functools.singledispatch).

The usual overloadable python operators has a dispatching function defined in `kdrag.notation`

In [39]:
kd.notation.add.register(Nat, add)
Nat.Zero + Nat.Zero

add(Zero, Zero)

For simple properties, again `kd.prove` suffices.

In [40]:
add_zero_one = kd.prove(Nat.Zero + one == one, by=[one.defn, add.defn])
add_zero_one

|- add(Zero, one) == one

In [41]:
add_zero_x = kd.prove(smt.ForAll([n], Nat.Zero + n == n), by=[add.defn])
add_zero_x

|- ForAll(n, add(Zero, n) == n)

In [42]:
add_two_x = kd.prove(smt.ForAll([n], add(two, n) == Nat.Succ(Nat.Succ(n))), by=[one.defn, two.defn, add.defn])
add_two_x

|- ForAll(n, add(two, n) == Succ(Succ(n)))

However, very similar properties seem out of reach for our particular definition of addition. In fact, they do not directly follow without an appeal to induction.

In [43]:
try:
    kd.prove(smt.ForAll([n], n + Nat.Zero == n), by=[add.defn], timeout=10)
except TimeoutError as e:
    print(e)

Timeout. Maybe you have given `prove` too many or not enough lemmas?


While you can directly use the auto generated induction axiom with `kd.prove`, it is easier to use the `kd.Lemma` interactive proof class.

This sort of thing is developed interactively. You can print `l` at any point to see the current goal and context.


In [44]:
l = kd.Lemma(smt.ForAll([n], n + Nat.Zero == n))
l

[] ?|- ForAll(n, add(n, Zero) == n)

We see that we have a quantified goal. The `intros` or `fixes` tactic will return a fresh list of variables and open the quantifier.

In [45]:
_n = l.fix()
l

[n!117] ; [] ?|- add(n!117, Zero) == n!117

For this proof, at this point we need to apply induction to `_n`. This returns the first induction case to be proved where `_n == Nat.zero`

In [46]:
l.induct(_n)

[n!117] ; [] ?|- add(Zero, Zero) == Zero

This case can easily be proven automatically given access to the definition. But we can also unfold the definition manually.

In [47]:
# would solve immiedateily l.auto(by=[add.defn]))
l.unfold(add)

[n!117];
[]
?|- If(is(Zero, Zero),
   Zero,
   If(is(Succ, Zero),
      Succ(add(pred(Zero), Zero)),
      unreachable!70)) ==
Zero

This can be simplified by z3. `is(Zero,Zero)` is `True` and the `If` will collapse.

In [48]:
# l.auto() would finish the goal here
l.simp()

[n!117] ; [] ?|- True

A goal of `True` is easily closed automatically with no extra needed information.

In [49]:
l.auto()

[n!117];
[]
?|- ForAll(pred!118,
       Implies(add(pred!118, Zero) == pred!118,
               add(Succ(pred!118), Zero) == Succ(pred!118)))

The next case is the inductive case.

In [50]:
l.auto(by=[add.defn])

Nothing to do!

To actually extract the proof object call a final `l.qed()`

In [51]:
add_x_zero = l.qed()
add_x_zero

|- ForAll(n, add(n, Zero) == n)


I have expanded these to multiple cells so that you can see the intermediate states, but I always do a proofs in a single cell.

In [52]:
l = kd.Lemma(smt.ForAll([n], n + Nat.Zero == n))
_n = l.fix()
l.induct(_n)
l.auto(by=[add.defn])
l.auto(by=[add.defn])
add_x_zero = l.qed()
add_x_zero

|- ForAll(n, add(n, Zero) == n)

### Exercise
Try to prove `add_x_succ`

In [53]:
l = kd.Lemma(smt.ForAll([n,m], n + Nat.Succ(m) == Nat.Succ(n + m)))
l
# TODO

[] ?|- ForAll([n, m], add(n, Succ(m)) == Succ(add(n, m)))

# References

- The Lean Natural Number Game <https://adam.math.hhu.de/>
- Software Foundations <https://softwarefoundations.cis.upenn.edu/lf-current/Induction.html>